<a href="https://colab.research.google.com/github/ppseguel/jobskillmatching/blob/master/industry_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import dataset
#follow this documentation, using Github, https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

import pandas as pd

url = 'https://raw.githubusercontent.com/ppseguel/jobskillmatching/master/marketingintern_industry.csv'
df1 = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

print(df1)

     Unnamed: 0                Other             Industryindex  \
0            42                  NaN                    Retail   
1           191                  NaN         Business Services   
2           249                  NaN                       NaN   
3            90                  NaN                      Auto   
4           149                  NaN  Energy, Manuf. & Utility   
5          2150                  NaN                       NaN   
6           276                  NaN     Media & Entertainment   
7           587  Commercial Services                       NaN   
8          2169                  NaN                       NaN   
9            19                  NaN                       NaN   
10           70                  NaN         Business Services   
11         1639                  NaN                       NaN   
12          842                  NaN                       NaN   
13          328                  NaN     Electronics & Telecom   
14        

In [0]:
#EXPERIMENT 1 (EDIT DATASET)


# 1. Construct Datasets for Training and Evaluation
# a) Load real dataset
from sklearn.datasets import load_wine

wine_data = load_wine()
X = wine_data.data
y = wine_data.target
print(wine_data.DESCR)

# b) Construct Datasets for Training and Evaluation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("Number samples in training: ", len(X_train))
print("Number samples in testing: ", len(X_test))




In [0]:
#EXPERIMENT 1 (EDIT DATASET AND TO MULTILABEL)

# a) Train and evaluate a decision treel model
from sklearn.tree import DecisionTreeClassifier

tree_giniIndex = DecisionTreeClassifier(criterion='entropy', max_depth=6)
tree_giniIndex.fit(X_train, y_train)

# Perform 10-fold cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=2)
tree_scores = cross_val_score(tree_giniIndex, X, y, cv=kfold, scoring="accuracy")
print("Cross-validation score:\n{}".format(tree_scores))
print("Average cross-validation score: {:.2f}".format(tree_scores.mean()))
print("Standard deviation: {:.2f}".format(tree_scores.std()))

# Tune Decision Tree hyperparameter using cross-validation
best_score = 0
for max_d in [1,3,5,7,9,11,13,15,17]:
    
    tree_giniIndex = DecisionTreeClassifier(max_depth=max_d).fit(X_train, y_train)
    fold_accuracies = cross_val_score(tree_giniIndex, X_train, y_train, cv=kfold, scoring="accuracy") 
    score = fold_accuracies.mean()
    
    print("Score for depth of " + str(max_d) + "on validation set is {:0.2f}".format(score))
    
    #Best score for best optimal model
    
    if score > best_score:
        best_param = {'max_depth': max_d}
        best_score = score
        
#Part of the answer of question 3        
tree_giniIndex = DecisionTreeClassifier(**best_param)
tree_giniIndex.fit(X_train, y_train)
test_score = tree_giniIndex.score(X_test, y_test)
print("Best score on cross-validation: {:0.2f}".format(best_score))
print("Best parameters: {}".format(best_param))
print("Test set score: {:.2f}".format(test_score))


In [0]:
#EXPERIMENT 1 (EDIT DATASET AND TO MULTILABEL)


# b) Nearest Neighbors (K-NN)
# Normalize data
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

# Tune hyperparameters using cross-validation for K-NN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=2)

best_score = 0

for p in [1, 2, 3, 4, 5]:
    for n in [1, 2, 3, 4, 5, 6]:   
        clf = KNeighborsClassifier(n_neighbors=n, p=p, metric='minkowski')
        clf.fit(X_train_norm, y_train)
            
        fold_accuracies = cross_val_score(clf, X_train, y_train, cv=kfold, scoring="accuracy")  
        score = fold_accuracies.mean()
        print("Score for p " + str(p) + ", score of neighbors " + str(n) + " on validation set is {:0.2f}".format(score))
    
        if score > best_score:
            best_param = {"p": p, "neighbors": n}
            best_score = score

#rebuild the model and evaluate the test score                
clf = KNeighborsClassifier(**best_param)
clf.fit(X_train_norm, y_train)
test_score = clf.score(X_test_norm, y_test)
print("Best score on cross-validation: {:0.2f}".format(best_score))
print("Best parameters: {}".format(best_param))
print("Test set score: {:.2f}".format(test_score))


In [0]:
#3 Optimal Decision Tree model predictive performance
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

tree_giniIndex = DecisionTreeClassifier(criterion='entropy', max_depth=3)
tree_giniIndex.fit(X_train, y_train)

y_predictedDT = tree_giniIndex.predict(X_test_scaled)
print(metrics.classification_report(y_predictedDT, y_test))
print(metrics.accuracy_score(y_predictedDT, y_test))

#3 Optimal SVM model predictive performance
from sklearn import metrics
from sklearn.svm import SVC

poly_kernel_svm_clf = SVC(kernel="poly", C = 1, degree=0, gamma=0.1)
poly_kernel_svm_clf.fit(X_train_scaled, y_train)
y_predictedSVM = poly_kernel_svm_clf.predict(X_test_scaled)
print(metrics.classification_report(y_predictedSVM, y_test))
print(metrics.accuracy_score(y_predictedSVM, y_test))


In [0]:
# Visualize confusion matrix for the Naive Bayes
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedNB, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')

# Visualize confusion matrix for Decision Tree
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedDT, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')

# Visualize confusion matrix for the SVM
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedSVM, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')


In [0]:
#MULTILABEL CONFUSION MATRIX
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_true, y_pred, sample_weight=None, labels=None, samplewise=False)



In [0]:
#EXPERIMENT 2
# Plot learning curves
def plot_learning_curves(model, x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
    train_errors, test_errors = [], []
   # change de len for m in range(1, len(x_train)):
    for m in range(1, 5):
        model.fit(x_train[:m], y_train[:m])
        y_train_predict = model.predict(x_train[:m])
        y_test_predict = model.predict(x_test)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        test_errors.append(mean_squared_error(y_test_predict, y_test))
        plt.plot(train_errors, "r-+", linewidth = 2, label = "Training Data")
        plt.plot(test_errors, "b-+", linewidth = 3, label = "Test Data")
        plt.ylabel("Mean squared error")
        plt.title("Learning Curves")
        plt.legend()
        
linear_reg_model = linear_model.LinearRegression()
plot_learning_curves(linear_reg_model, x_real, y_real)